In [46]:
import datetime
from functools import reduce
import locale
import json
import pandas as pd

In [70]:
df = pd.read_json('../../data/raw/awl.json')


### Added date

In [72]:
# set locale to extract month number from its portuguese name
locale.setlocale(locale.LC_TIME, "pt_BR")
pattern = 'Item\ adicionado\ (..?)\ de\ (\w*)\ de\ (....)'

def concat_date(x, y):
  return '/'.join([str(x), str(datetime.datetime.strptime(y, '%B').month if len(x) <= 2 else str(y))])

df['added_date'] = [reduce(concat_date, row) for row in df['added_date'].str.extract(pattern).values.tolist()]

### Review stars

In [74]:
pattern = '(.\..)\ .*'
df['review_stars'] = df['review_stars'].str.extract(pattern)

### Total price

In [97]:
# neither sold by Amazon nor external sellers
filter = (df['price'].isna()) & (df['sellers_price'].isna())
df.loc[filter, 'total_price'] = -1

# only sold by external sellers (might apply not tracked delivery tax)
filter = (df['price'].isna()) & (~df['sellers_price'].isna())
df.loc[filter, 'total_price'] = df['sellers_price']

# sold by Amazon without free shipping
filter = (~df['price'].isna()) & (~df['delivery_price'].isna())
df.loc[filter, 'total_price'] = df['price'] + df['delivery_price']

# sold by Amazon with free shipping
filter = (~df['price'].isna()) & (df['delivery_price'].isna())
df.loc[filter, 'total_price'] = df['price']

### Availability

In [99]:
# df['availability'].loc[~df['availability'].isnull()] = 1
# df['availability'].fillna(0, inplace=True)
df['availability'] = 1
filter = (df['total_price'] == -1.0)
df.loc[filter, 'availability'] = 0

### Reference date

In [101]:
df['reference_date'] = datetime.datetime.now()


### Output csv

In [102]:
df.to_csv('../../data/preparation/preparation.csv', sep=';', index=False, mode='a', header='false')